https://autokeras.com/tutorial/text_classification/



In [ ]:
!pip install autokeras

In [2]:
import os
import numpy as np
import tensorflow as tf
from sklearn.datasets import load_files
import autokeras as ak

Using TensorFlow backend


In [3]:
dataset = tf.keras.utils.get_file(
    fname="aclImdb.tar.gz",
    origin="http://ai.stanford.edu/~amaas/data/sentiment/aclImdb_v1.tar.gz",
    extract=True,
)
IMDB_DATADIR = os.path.join(os.path.dirname(dataset), "aclImdb")
train_data = load_files(
    os.path.join(IMDB_DATADIR, "train"), shuffle=True, categories=["pos", "neg"]
)
test_data = load_files(
    os.path.join(IMDB_DATADIR, "test"), shuffle=False, categories=["pos", "neg"]
)

84125825/84125825 [==============================] - 16s 0us/step


In [4]:
# train_data.data is a list
train = np.array(train_data.data)
test = np.array(test_data.data)
# y_test = np.array(test_data.target) TODO: why was it like this?

print("train shape:", train.shape)
print("test shape:", test.shape)


train shape: (25000,)
test shape: (25000,)


In [ ]:
# TODO: What does max_trials mean?
clf = ak.TextClassifier(
    # overwrite=True,
    # max_trials=1
)
clf.fit(
    x=train,
    y=test,
    # steps_per_epoch=32,
    batch_size=16,
    # epochs=10
)
print(clf.evaluate(x=train, y=test))


Trial 2 Complete [00h 06m 12s]
val_loss: 10.6158447265625

Best val_loss So Far: 10.570926666259766
Total elapsed time: 00h 11m 52s

Search: Running Trial #3

Value             |Best Value So Far |Hyperparameter
bert              |vanilla           |text_block_1/block_type
0                 |0                 |classification_head_1/dropout
adam_weight_decay |adam              |optimizer
2e-05             |0.001             |learning_rate
512               |None              |text_block_1/bert_block_1/max_sequence_length



100%|██████████| 547/547 [00:00<00:00, 819kB/s]
100%|██████████| 226k/226k [00:00<00:00, 683kB/s]
100%|██████████| 510/510 [00:00<00:00, 592kB/s]
100%|██████████| 418M/418M [00:11<00:00, 36.7MB/s]
/usr/local/lib/python3.10/dist-packages/keras_nlp/src/models/backbone.py:46: UserWarning: `Model.state_updates` will be removed in a future version. This property should not be used in TensorFlow 2.0, as `updates` are applied automatically.
  return id(getattr(self, attr)) not in self._functional_layer_ids
/usr/local/lib/python3.10/dist-packages/keras_nlp/src/models/backbone.py:46: UserWarning: `layer.updates` will be removed in a future version. This property should not be used in TensorFlow 2.0, as `updates` are applied automatically.
  return id(getattr(self, attr)) not in self._functional_layer_ids


Epoch 1/1000
Not enough memory, reduce batch size to 8.
Epoch 1/1000
   1704/Unknown - 629s 361ms/step - loss: 10.1198 - accuracy: 0.0000e+00

In [ ]:
clf.predict(np.array(["it was amazing! the acting was perfect!"]))

In [ ]:
input_node = ak.TextInput()
output_node = ak.TextBlock(block_type="ngram")(input_node)
output_node = ak.ClassificationHead()(output_node)
clf = ak.AutoModel(
    inputs=input_node, outputs=output_node, overwrite=True, max_trials=1
)
clf.fit(train, test, epochs=2)


In [ ]:
input_node = ak.TextInput()
output_node = ak.TextToIntSequence()(input_node)
output_node = ak.Embedding()(output_node)
# Use separable Conv layers in Keras.
output_node = ak.ConvBlock(separable=True)(output_node)
output_node = ak.ClassificationHead()(output_node)
clf = ak.AutoModel(
    inputs=input_node, outputs=output_node, overwrite=True, max_trials=1
)
clf.fit(train, test, epochs=2)


In [ ]:
train_set = tf.data.Dataset.from_tensor_slices(((train,), (test,))).batch(32)
test_set = tf.data.Dataset.from_tensor_slices(((train,), (test,))).batch(32)

clf = ak.TextClassifier(overwrite=True, max_trials=2)
# Feed the tensorflow Dataset to the classifier.
clf.fit(train_set, epochs=2)
# Predict with the best model.
predicted_y = clf.predict(test_set)
# Evaluate the best model with testing data.
print(clf.evaluate(test_set))
